In [2]:
import ee
import os
import geojson
#import matplotlib.pyplot as plt
#import numpy as np
import IPython.display as disp
#from geemap import png_to_gif
#from geemap.cartoee import get_map, add_gridlines, add_scale_bar_lite, add_north_arrow
import requests
import time
#ee.Authenticate()
ee.Initialize()
%matplotlib inline

In [20]:
filename = "gibraltar.geojson"
with open("Locations\\" + filename) as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


start_date = "2021-01-01" # 
end_date = "2021-12-31" #

ffa_db = (ee.ImageCollection('COPERNICUS/S1_GRD')
                    .filter(ee.Filter.eq('instrumentMode', 'IW'))
                    .filter(ee.Filter.eq('resolution_meters', 10))
                    .filterBounds(aoi)
                    .filterDate(ee.Date(start_date), ee.Date(end_date))
                    .sort('system:time_start'))


im_list = ffa_db.toList(ffa_db.size())
size = ffa_db.size().getInfo()


In [17]:
mode = "VV"

if mode == "VV":
    min_vision = -25
    max_vision = 10
elif mode == "VH":
    min_vision = -20
    max_vision = 0

acq_times = ffa_db.aggregate_array('system:time_start').getInfo()
dates = [time.strftime("-%Y%m%d-%H%M%S", time.gmtime(acq_time/1000)) for acq_time in acq_times]

len(dates)

253

In [18]:
url = ee.Image(im_list.get(45)).select(mode).clip(aoi).getThumbURL({'min': min_vision, 'max': max_vision})
disp.Image(url=url, width=1200)